In [ ]:
import numpy as np
import plotly.graph_objects as go

from numpy.linalg import norm as l2
from scipy.optimize import curve_fit, minimize

In [ ]:
alpha = np.random.rand()
beta = np.random.rand()
noise = np.random.normal(0, 1, 100)
x = np.linspace(0, 1, 101)
data = alpha * x + beta
y = data + np.random.normal(0, 1, 101)

range_ = [[0, 1], [0, 1]]
epsilon = 1e-3
initial_guess = np.array([0, 0])

print(f'alpha: {alpha}, beta: {beta}')

alpha: 0.26665962604585847, beta: 0.13817869152635942


#### Linear Approximation

In [ ]:
def linear_approximant(x, a, b):
    return a * x + b

In [ ]:
def cost_function(parameters, approximant, x, y):
    return np.sum((approximant(x, parameters[0], parameters[1]) - y) ** 2)

In [ ]:
def linear_gradient_descent(x, y, weights, learning_rate=10e-4, max_n_iterations=10000, epsilon=1e-3):
    n_iterations = 0
    n = y.size
    x = np.c_[ x, np.ones(n) ]
    previous_weights = np.array([1, 1])
    while (max_n_iterations > n_iterations) and l2(weights - previous_weights) > epsilon:
        previous_weights = weights
        prediction = np.dot(x, weights)
        df_weights = 2 * (x.T.dot(prediction - y)) / n
        weights = weights - learning_rate * df_weights
        n_iterations += 1
    print(f'alpha: {weights[0]:.5f}\n'
          f'beta: {weights[1]:.5f}\n'
          f'iterations: {n_iterations}')
    return weights

In [ ]:
linear_gd_weights = linear_gradient_descent(x, y, initial_guess)

alpha: 0.00022
beta: 0.00063
iterations: 1


Conjugate Gradient Descent

In [ ]:
weights_gd_linear = minimize(cost_function,
                             x0=initial_guess,
                             method='CG',
                             tol=epsilon,
                             args=(linear_approximant, x, y))
print(f'estimated alpha: {weights_gd_linear["x"][0]:.5f}\n'
      f'estimated beta: {weights_gd_linear["x"][1]:.5f}\n'
      f'iterations: {weights_gd_linear["nit"]}\n'
      f'function evaluations: {weights_gd_linear["nfev"]}')

estimated alpha: -0.58946
estimated beta: 0.61069
iterations: 2
function evaluations: 20


Newton’s method

In [ ]:
weights_newton_linear = minimize(cost_function,
                                 x0=initial_guess,
                                 method='BFGS',
                                 tol=epsilon,
                                 args=(linear_approximant, x, y))
print(f'estimated alpha: {weights_newton_linear["x"][0]:.5f}\n'
      f'estimated beta: {weights_newton_linear["x"][1]:.5f}\n'
      f'iterations: {weights_newton_linear["nit"]}\n'
      f'function evaluations: {weights_newton_linear["nfev"]}')

estimated alpha: -0.58946
estimated beta: 0.61069
iterations: 2
function evaluations: 20


Levenberg-Marquardt algorithm

In [ ]:
weights_linear_lm, _ = curve_fit(linear_approximant, x, y, p0=initial_guess, method='lm')
print(f'estimated alpha: {weights_linear_lm[0]:.5f}\n'
      f'estimated beta: {weights_linear_lm[1]:.5f}')

estimated alpha: -0.58946
estimated beta: 0.61069


In [ ]:
figure = go.Figure()
figure.add_trace(go.Scatter(x=x, y=data,
                            mode='lines',
                            name='generative line'))
figure.add_trace(go.Scatter(x=x, y=linear_gd_weights[0] * x + linear_gd_weights[1],
                            mode='lines',
                            name='linear gradient descent approximation'))
figure.add_trace(go.Scatter(x=x, y=weights_gd_linear["x"][0] * x + weights_gd_linear["x"][1],
                            mode='lines',
                            name='linear conjugate gradient descent approximation'))
figure.add_trace(go.Scatter(x=x, y=weights_newton_linear["x"][0] * x + weights_newton_linear["x"][1],
                            mode='lines',
                            name='linear quasi-newton approximation'))
figure.add_trace(go.Scatter(x=x, y=weights_linear_lm[0] * x + weights_linear_lm[1],
                            mode='lines',
                            name='linear levenberg-marquardt approximation'))
figure.add_trace(go.Scatter(x=x, y=y,
                            mode='markers',
                            name='noised data'))
figure.update_layout(title='Linear Approximation', font_size=10)

In [ ]:
n_iter = f calls
compare with other

#### Rational approximation

In [ ]:
def rational_approximant(x, a, b):
    return a / (1 + b * x)

Gradient Descent

In [ ]:
def rational_gd(x, y, a, b, learning_rate=10e-4, max_n_iterations=10000, epsilon=1e-3):
    n_iterations = 0
    n = y.size
    previous_weights = np.array([1, 1])
    while (max_n_iterations > n_iterations) and l2(np.array([a, b]) - previous_weights) > epsilon:
        previous_weights = np.array([a, b])
        df_a = 2 * np.sum((a * (1 + b * x) ** (-1) - y) * a * (1 + b * x) ** (-1)) / n
        df_b = 2 * np.sum((a * (1 + b * x) ** (-1) - y) * ( - a) * x * (1 + b * x) ** (-2)) / n
        a = a - learning_rate * df_a
        b = b - learning_rate * df_b
        n_iterations += 1
    print(f'alpha: {a:.5f}\n'
          f'beta: {b:.5f}\n'
          f'iterations: {n_iterations}')
    return np.array([a, b])

In [ ]:
rational_gd_weights = linear_gradient_descent(x, y, [0.1, 0.1])

alpha: 0.10005
beta: 0.10033
iterations: 1


Conjugate Gradient Descent

In [ ]:
weights_cg_rational = minimize(cost_function,
                              x0=initial_guess,
                              method='CG',
                              tol=epsilon,
                              args=(rational_approximant, x, y))
print(f'estimated alpha: {weights_cg_rational["x"][0]:.5f}\n'
      f'estimated beta: {weights_cg_rational["x"][1]:.5f}\n'
      f'iterations: {weights_cg_rational["nit"]}\n'
      f'function evaluations: {weights_cg_rational["nfev"]}')

estimated alpha: 0.78705
estimated beta: 3.80726
iterations: 8
function evaluations: 76


Newton’s method

In [ ]:
weights_newton_rational = minimize(cost_function,
                                 x0=initial_guess,
                                 method='BFGS',
                                 tol=epsilon,
                                 args=(rational_approximant, x, y))
print(f'estimated alpha: {weights_newton_rational["x"][0]:.5f}\n'
      f'estimated beta: {weights_newton_rational["x"][1]:.5f}\n'
      f'iterations: {weights_newton_rational["nit"]}\n'
      f'function evaluations: {weights_newton_rational["nfev"]}')

estimated alpha: 0.78698
estimated beta: 3.80640
iterations: 11
function evaluations: 52


Levenberg-Marquardt algorithm

In [ ]:
weights_rational_lm, _ = curve_fit(rational_approximant, x, y, p0=initial_guess, method='lm')
print(f'estimated alpha: {weights_rational_lm[0]:.5f}\n'
      f'estimated beta: {weights_rational_lm[1]:.5f}')

estimated alpha: 0.78708
estimated beta: 3.80767


In [ ]:
figure = go.Figure()
figure.add_trace(go.Scatter(x=x, y=data,
                            mode='lines',
                            name='generative line'))
figure.add_trace(go.Scatter(x=x, y=rational_gd_weights[0] / (1 + rational_gd_weights[1] * x),
                            mode='lines',
                            name='rational gradient descent approximation'))
figure.add_trace(go.Scatter(x=x, y=weights_cg_linear["x"][0] / (1 + rational_cg_linear["x"][1] * x),
                            mode='lines',
                            name='rational conjugate gradient descent approximation'))
figure.add_trace(go.Scatter(x=x, y=weights_cg_rational["x"][0] / + (1 + weights_cg_rational["x"][1] * x),
                            mode='lines',
                            name='rational quasi-newton approximation'))
figure.add_trace(go.Scatter(x=x, y=weights_rational_lm[0] / (1 + weights_rational_lm[1] * x),
                            mode='lines',
                            name='linear levenberg-marquardt approximation'))
figure.add_trace(go.Scatter(x=x, y=y,
                            mode='markers',
                            name='noised data'))
figure.update_layout(title='Rational Approximation', font_size=10)